# Assignment 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../Assignment1_2020.xlsx', header = [0,1])[:12]
data

Unnamed: 0_level_0                 Single70                            \
               Months Sales\nForecast\n(Units) Production\nCost ($/unit)   
0                 Jan                  25000.0                     450.0   
1                 Feb                  25000.0                     450.0   
2                 Mar                  26000.0                     450.0   
3                 Apr                  26000.0                     475.0   
4                 May                  27000.0                     475.0   
5                 Jun                  27000.0                     475.0   
6                 Jul                  28000.0                     500.0   
7                 Aug                  29000.0                     500.0   
8                 Sep                  30000.0                     500.0   
9                 Oct                  30000.0                     400.0   
10                Nov                  30000.0                     400.0   
11                Dec                  30000.0                     400.0   

                   Double70                            \
   Sales\nForecast\n(Units) Production\nCost ($/unit)   
0                   18500.0                     650.0   
1                   19000.0                     650.0   
2                   20000.0                     650.0   
3                   21000.0                     625.0   
4                   20000.0                     600.0   
5                   19000.0                     590.0   
6                   18000.0                     580.0   
7                   17000.0                     550.0   
8                   19000.0                     550.0   
9                   20000.0                     550.0   
10                  20500.0                     610.0   
11                  21000.0                     640.0   

                   Double80                            
   Sales\nForecast\n(Units) Production\nCost ($/unit)  
0                    9500.0                     810.0  
1                    9500.0                     810.0  
2                    9500.0                     880.0  
3                    7500.0                     880.0  
4                    7500.0                     880.0  
5                    7500.0                     880.0  
6                    6500.0                     880.0  
7                    6500.0                     840.0  
8                    6500.0                     840.0  
9                    6500.0                     840.0  
10                   6500.0                     880.0  
11                   6500.0                     880.0

In [3]:
data = data.rename(columns={"Sales\nForecast\n(Units)":"SalesForecast", "Production\nCost ($/unit)":"ProductionCost"})
data

Unnamed: 0_level_0      Single70                     Double70  \
               Months SalesForecast ProductionCost SalesForecast   
0                 Jan       25000.0          450.0       18500.0   
1                 Feb       25000.0          450.0       19000.0   
2                 Mar       26000.0          450.0       20000.0   
3                 Apr       26000.0          475.0       21000.0   
4                 May       27000.0          475.0       20000.0   
5                 Jun       27000.0          475.0       19000.0   
6                 Jul       28000.0          500.0       18000.0   
7                 Aug       29000.0          500.0       17000.0   
8                 Sep       30000.0          500.0       19000.0   
9                 Oct       30000.0          400.0       20000.0   
10                Nov       30000.0          400.0       20500.0   
11                Dec       30000.0          400.0       21000.0   

                       Double80                 
   ProductionCost SalesForecast ProductionCost  
0           650.0        9500.0          810.0  
1           650.0        9500.0          810.0  
2           650.0        9500.0          880.0  
3           625.0        7500.0          880.0  
4           600.0        7500.0          880.0  
5           590.0        7500.0          880.0  
6           580.0        6500.0          880.0  
7           550.0        6500.0          840.0  
8           550.0        6500.0          840.0  
9           550.0        6500.0          840.0  
10          610.0        6500.0          880.0  
11          640.0        6500.0          880.0

In [4]:
inv_car = np.r_[[0.01]*6,[0.013]*6]
prod_time = np.r_[1.65,2.15,2.68]
reg_wf = np.r_[[90000]*6,[100000]*6]
overtime_wf = reg_wf*0.25
reg_cost = np.r_[[30]*6,[35]*6]
overtime_cost = reg_cost*1.5
init_inv = np.r_[6500,12000,3000]

In [5]:
prod_cost = pd.concat([data.Single70.ProductionCost, data.Double70.ProductionCost, data.Double80.ProductionCost],axis=1).values
sales_forecast = pd.concat([data.Single70.SalesForecast, data.Double70.SalesForecast, data.Double80.SalesForecast],axis=1).values

IC = inv_car.reshape((12,1))*prod_cost

## Base Model

In [6]:
import pyomo.environ as pyo

p = 3
t=12

objRule = lambda model : sum(model.X[t,p]*prod_cost[t,p] for t in model.t for p in model.p) + sum(model.I[t+1,p]*IC[t,p] for t in model.t for p in model.p) + sum(model.W[t]*reg_cost[t] for t in model.t) + sum(model.O[t]*overtime_cost[t] for t in model.t)

# Inv holding
c0 = lambda model, t, p : model.X[t,p]+model.I[t,p]-sales_forecast[t,p] == model.I[t+1,p]
# No shortage constraint is also applied with nonnegative inventory

# Production-workforce time
c1 = lambda model, t: sum(model.X[t,p]*prod_time[p] for p in model.p) <= model.W[t] + model.O[t]

# Overtime constraint
c2 = lambda model, t: model.O[t] <= overtime_wf[t]

# Initial inv
c3 = lambda model, p : model.I[0,p] == init_inv[p]

# Regular workforce:
c4 = lambda model, t : model.W[t] <= reg_wf[t]

def constructBaseModel(model):
    model.p = range(p)
    model.t = range(t)
    model.t2 = range(t+1)

    model.X = pyo.Var(model.t, model.p, domain=pyo.NonNegativeReals)
    model.I = pyo.Var(model.t2, model.p, domain=pyo.NonNegativeReals)
    model.W = pyo.Var(model.t, domain=pyo.NonNegativeReals)
    model.O = pyo.Var(model.t, domain=pyo.NonNegativeReals)
    
    model.obj = pyo.Objective(rule = objRule, sense=pyo.minimize)
    
    model.c0 = pyo.Constraint(model.t, model.p, rule=c0)
    model.c1 = pyo.Constraint(model.t, rule=c1)
    model.c2 = pyo.Constraint(model.t, rule=c2)
    model.c3 = pyo.Constraint(model.p, rule=c3)
    model.c4 = pyo.Constraint(model.t, rule=c4)

In [7]:
base = pyo.ConcreteModel()
constructBaseModel(base)

opt = pyo.SolverFactory('cbc')
opt.solve(base, tee = True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmptj2aib__.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmptj2aib__.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 45 (-31) rows, 93 (-7) columns and 159 (-37) elements
Statistics for presolved model


Problem has 45 rows, 93 columns (93 with objective) and 159 elements
There are 27 singletons with objective 
Column breakdown:
66 of type 0.0->inf, 24 of type 0.0->up, 3 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
33 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0

{'Problem': [{'Name': 'unknown', 'Lower bound': 395706458.8, 'Upper bound': 395706458.8, 'Number of objectives': 1, 'Number of constraints': 76, 'Number of variables': 100, 'Number of nonzeros': 93, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 73}}, 'Error rc': 0, 'Time': 0.0428006649017334}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
base.obj()

395706458.61730003

## Extension 1

In [9]:
bigM = 1000000

extra_wf = np.zeros((12,))
extra_wf[5:8] = bigM
extra_cost = 38

In [10]:
p=3
t=12

# Update objective
ex1Obj = lambda model : objRule(model) + sum(model.E[t]*extra_cost for t in model.t)

# Extra workforce constraint
c5 = lambda model, t : model.E[t] <= extra_wf[t]

# Production-workforce constraint update
c1_upd = lambda model, t: sum(model.X[t,p]*prod_time[p] for p in model.p) <= model.W[t] + model.O[t] + model.E[t]

def constructExtension1(model):
    model.p = range(p)
    model.t = range(t)
    model.t2 = range(t+1)

    model.X = pyo.Var(model.t, model.p, domain=pyo.NonNegativeReals)
    model.I = pyo.Var(model.t2, model.p, domain=pyo.NonNegativeReals)
    model.W = pyo.Var(model.t, domain=pyo.NonNegativeReals)
    model.O = pyo.Var(model.t, domain=pyo.NonNegativeReals)
    model.E = pyo.Var(model.t, domain=pyo.NonNegativeReals)
    
    model.obj = pyo.Objective(rule = ex1Obj, sense=pyo.minimize)
    
    model.c0 = pyo.Constraint(model.t, model.p, rule=c0)
    model.c1_upd = pyo.Constraint(model.t, rule=c1_upd)
    model.c2 = pyo.Constraint(model.t, rule=c2)
    model.c3 = pyo.Constraint(model.p, rule=c3)
    model.c4 = pyo.Constraint(model.t, rule=c4)
    model.c5 = pyo.Constraint(model.t, rule=c5)    

In [11]:
ext1 = pyo.ConcreteModel()
constructExtension1(ext1)

opt.solve(ext1, tee = True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmp201c8onk.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmp201c8onk.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 45 (-43) rows, 96 (-16) columns and 162 (-58) elements
Statistics for presolved model


Problem has 45 rows, 96 columns (96 with objective) and 162 elements
There are 30 singletons with objective 
Column breakdown:
66 of type 0.0->inf, 27 of type 0.0->up, 3 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
33 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 

{'Problem': [{'Name': 'unknown', 'Lower bound': 394054275.6, 'Upper bound': 394054275.6, 'Number of objectives': 1, 'Number of constraints': 88, 'Number of variables': 112, 'Number of nonzeros': 96, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 60}}, 'Error rc': 0, 'Time': 0.06683731079101562}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
ext1.obj()

394054275.84337497

In [13]:
base.obj() - ext1.obj()

1652182.773925066

## Extension 2

In [14]:
inv_cap = 25000

In [15]:
p=3
t=12

# Max inventory constraint
ex2_1 = lambda model, t: sum(model.I[t,p] for p in model.p) <= inv_cap

def constructExtension2(model):
    constructBaseModel(model)
    model.ex2_1 = pyo.Constraint(model.t, rule=ex2_1)

In [16]:
ext2 = pyo.ConcreteModel()
constructExtension2(ext2)

opt.solve(ext2, tee=True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmpedvku4pj.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmpedvku4pj.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 56 (-32) rows, 93 (-7) columns and 192 (-40) elements
Statistics for presolved model


Problem has 56 rows, 93 columns (93 with objective) and 192 elements
There are 27 singletons with objective 
Column breakdown:
66 of type 0.0->inf, 24 of type 0.0->up, 3 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
33 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
11 of type L other, 0 of type Range 0.0->1.0, 

{'Problem': [{'Name': 'unknown', 'Lower bound': 396131181.2, 'Upper bound': 396131181.2, 'Number of objectives': 1, 'Number of constraints': 88, 'Number of variables': 100, 'Number of nonzeros': 93, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 71}}, 'Error rc': 0, 'Time': 0.05984044075012207}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [17]:
ext2.obj()

396131179.75717

In [18]:
base.obj() - ext2.obj()

-424721.13986998796

## Extension 3

In [19]:
p=3
t=12

def constructExtension3(model):
    constructExtension1(model)
    model.ex2_1 = pyo.Constraint(model.t, rule=ex2_1)

In [20]:
ext3 = pyo.ConcreteModel()
constructExtension3(ext3)

opt.solve(ext3, tee=True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmp7ub_tzp0.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmp7ub_tzp0.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 56 (-44) rows, 96 (-16) columns and 195 (-61) elements
Statistics for presolved model


Problem has 56 rows, 96 columns (96 with objective) and 195 elements
There are 30 singletons with objective 
Column breakdown:
66 of type 0.0->inf, 27 of type 0.0->up, 3 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
33 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
11 of type L other, 0 of type Range 0.0->1.0,

{'Problem': [{'Name': 'unknown', 'Lower bound': 395584496.9, 'Upper bound': 395584496.9, 'Number of objectives': 1, 'Number of constraints': 100, 'Number of variables': 112, 'Number of nonzeros': 96, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 75}}, 'Error rc': 0, 'Time': 0.06881403923034668}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [21]:
ext3.obj()

395584496.69719493

In [22]:
base.obj() - ext3.obj()

121961.92010509968

In [23]:
ext1.obj() - ext3.obj()

-1530220.8538199663

In [24]:
ext2.obj() - ext3.obj()

546683.0599750876

## Extension 4

In [25]:
inv_car_updated = np.r_[[0.01]*6,[0.02]*6]
IC_updated = inv_car_updated.reshape((12,1))*prod_cost

In [26]:
p=3
t=12

ext4_obj = lambda model : sum(model.X[t,p]*prod_cost[t,p] for t in model.t for p in model.p) + sum(model.I[t+1,p]*IC_updated[t,p] for t in model.t for p in model.p) + sum(model.W[t]*reg_cost[t] for t in model.t) + sum(model.O[t]*overtime_cost[t] for t in model.t)

def constructExtension4(model):
    constructBaseModel(model)
    model.del_component(model.obj)
    
    model.obj = pyo.Objective(rule=ext4_obj, sense=pyo.minimize)

In [27]:
ext4 = pyo.ConcreteModel()
constructExtension4(ext4)

opt.solve(ext4, tee=True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmpskm5njk9.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmpskm5njk9.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 45 (-31) rows, 93 (-7) columns and 159 (-37) elements
Statistics for presolved model


Problem has 45 rows, 93 columns (93 with objective) and 159 elements
There are 27 singletons with objective 
Column breakdown:
66 of type 0.0->inf, 24 of type 0.0->up, 3 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
33 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0

{'Problem': [{'Name': 'unknown', 'Lower bound': 396368861.5, 'Upper bound': 396368861.5, 'Number of objectives': 1, 'Number of constraints': 76, 'Number of variables': 100, 'Number of nonzeros': 93, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 67}}, 'Error rc': 0, 'Time': 0.06781601905822754}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [28]:
ext4.obj()

396368861.31595

In [29]:
base.obj() - ext4.obj()

-662402.6986499429

## Extension 5

In [30]:
import itertools as it

In [31]:
cases = list(it.product([0.9,1.1],repeat=3))

In [32]:
p=3
t=12

def constructExtension5(model, case):
    constructExtension2(model)
    model.del_component(model.c1)
    c1_updated = lambda model, t: sum(model.X[t,p]*prod_time[p]*case[p] for p in model.p) <= model.W[t] + model.O[t]
    model.c1_upd = pyo.Constraint(model.t, rule=c1_updated)

In [33]:
objectives = []
for c in cases:
    ext5 = pyo.ConcreteModel()
    constructExtension5(ext5, c)
    opt.solve(ext5)
    objectives.append(ext5.obj())

In [34]:
for i in range(len(cases)):
    print("Case {0} -> Objective : {1}\t-> Base-New : {2}".format(cases[i],objectives[i],base.obj()-objectives[i]))

Case (0.9, 0.9, 0.9) -> Objective : 390512284.75321996	-> Base-New : 5194173.864080071
Case (0.9, 0.9, 1.1) -> Objective : 392612661.2126	-> Base-New : 3093797.404700041
Case (0.9, 1.1, 0.9) -> Objective : 394735107.78910005	-> Base-New : 971350.8281999826
Case (0.9, 1.1, 1.1) -> Objective : 397061021.5492749	-> Base-New : -1354562.9319748878
Case (1.1, 0.9, 0.9) -> Objective : 395179299.1948701	-> Base-New : 527159.4224299192
Case (1.1, 0.9, 1.1) -> Objective : 397580235.86978	-> Base-New : -1873777.2524799705
Case (1.1, 1.1, 0.9) -> Objective : 400063669.36012006	-> Base-New : -4357210.7428200245
Case (1.1, 1.1, 1.1) -> Objective : 403033533.00425494	-> Base-New : -7327074.386954904


## Extension 6

In [36]:
sales_updated = sales_forecast.copy()
sales_updated[3:6,2] = 17500

In [37]:
p=3
t=12

bigM = 100000

c0_updated = lambda model, t, p : model.X[t,p]+model.I[t,p]-sales_updated[t,p] == model.I[t+1,p] - model.S[t,p]
ext6_obj = lambda model : objRule(model) + sum(model.S[t,p]*bigM for t in model.t for p in model.p)

def constructExtension6(model):
    constructExtension2(model)
    
    # Add shortage variable
    model.S = pyo.Var(model.t, model.p, domain=pyo.NonNegativeReals)
    
    model.del_component(model.c0)
    model.del_component(model.obj)
    
    model.obj = pyo.Objective(rule=ext6_obj, sense=pyo.minimize)
    model.c0_upd = pyo.Constraint(model.t, model.p, rule=c0_updated)

In [38]:
ext6 = pyo.ConcreteModel()
constructExtension6(ext6)

opt.solve(ext6, tee=True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmpzv1gb0s7.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmpzv1gb0s7.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 59 (-29) rows, 132 (-4) columns and 234 (-34) elements
Statistics for presolved model


Problem has 59 rows, 132 columns (132 with objective) and 234 elements
There are 63 singletons with objective 
Column breakdown:
108 of type 0.0->inf, 24 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
36 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
11 of type L other, 0 of type Range 0.0->1

{'Problem': [{'Name': 'unknown', 'Lower bound': 705328606.2, 'Upper bound': 705328606.2, 'Number of objectives': 1, 'Number of constraints': 88, 'Number of variables': 136, 'Number of nonzeros': 132, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 85}}, 'Error rc': 0, 'Time': 0.05834460258483887}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [39]:
shortage = np.zeros((ext6.t.stop,ext6.p.stop))
for t in ext6.t:
    for p in ext6.p:
        shortage[t,p] = ext6.S[t,p]()
shortage

array([[   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    , 2813.4328],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ],
       [   0.    ,    0.    ,    0.    ]])